In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import spacy
from spacy import displacy
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"

2023-06-07 14:29:53.571996: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 14:29:54.264912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-07 14:29:54.920156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 14:29:54.921043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [60]:
def verifyDate(date):
    dateList = []
    for i in range(len(date)):
        dateList.append(date[i])
    symbols = [":", "/", "\\", "-", ",", ".", " ", "  "]
    while len(dateList)!=0 and dateList[0] in symbols:
        dateList.pop(0)
    while len(dateList)!=0 and dateList[-1] in symbols:
        dateList.pop(-1)
    count = 0
    for symbol in  symbols:
        if symbol in dateList:
            count += dateList.count(symbol)
    if count!=2:
        return False
    else:
        return True

In [71]:
def preProcessText(text):
    symbols = [":", "/", "\\", "-", ",", ".", " ", "  ", "   "]
    months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "sept", "oct", "nov", "dec", "january"
             , "february", "march", "april", "june", "july", "august", "september", "october", "november", "december"]
    monthDict = {
        "j" : "01",
        "f" : "02",
        "m" : "03",
        "a" : "04",
        "j" : "07",
        "s" : "09",
        "o" : "10",
        "n" : "11",
        "d" : "12"
    }
    text=text.lower()
    if verifyDate(text):
        for symbol in symbols:
            if symbol in text:
                text = text.replace(symbol, "/")
        L = text.split("/")
        processedDate = []
        for t in L:
            if t.lower() in months:
                if t.lower()=="may":
                    encoding = "05"
                elif t.lower()=="aug" or t.lower()=="august":
                    encoding = "08"
                elif t.lower()=="jun" or t.lower()=="june":
                    encoding = "06"
                else:
                    encoding = monthDict[t[0].lower()]
                processedDate.append(encoding)
            else:
                processedDate.append(t)
        if len(processedDate[-1])==2:
            processedDate[-1] = "20"+processedDate[-1]
        date = "-".join(processedDate)
        return date
    else:
        return text

In [72]:
listOfDates = ["5/12/22", "2/2/2023", "5:12-2023", "5,May,2023", "5 May,2023", "October 24,2022"]

In [73]:
print("Original Date", "---->", "After Processing", "\n")
for date in listOfDates:
    processed = preProcessText(date)
    print(date, "     ---->     ", processed)

Original Date ----> After Processing 

5/12/22      ---->      5-12-2022
2/2/2023      ---->      2-2-2023
5:12-2023      ---->      5-12-2023
5,May,2023      ---->      5-05-2023
5 May,2023      ---->      5-05-2023
October 24,2022      ---->      10-24-2022
